In [69]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup, Comment
import pandas as pd
import numpy as np
import datetime
import unidecode
from nba import NbaTracker
import time
from nba_api.stats import static
import requests
from dataclasses import dataclass, field



In [3]:
from nba import NbaTracker
seasons = [
    "2000-01",
    "2001-02",
    "2002-03",
    "2003-04",
    "2004-05",
    "2005-06",
    "2006-07",
    "2007-08",
    "2008-09",
    "2009-10",
    "2010-11",
    "2011-12",
    "2012-13",
    "2013-14",
    "2014-15",
    "2015-16",
    "2016-17",
    "2017-18",
    "2018-19",
    "2019-20",
    "2020-21"
]
nbaTracker = NbaTracker()
for season_name in seasons[:]:
    nbaTracker.add_season(season_name)


No missing ids
No missing ids
No missing ids
No missing ids
No missing ids
No missing ids
No missing ids
No missing ids
No missing ids
No missing ids
No missing ids
No missing ids
No missing ids
No missing ids
No missing ids
No missing ids
No missing ids
No missing ids
No missing ids
No missing ids
No missing ids


In [4]:
season = nbaTracker.current_season



In [6]:
from nba_api.stats.static.teams import find_team_name_by_id
def get_team_abr(team_id, season_name):
    if team_id == 1610612740:  # NOP
        if season_name <= "2004-05":
            return "NOH"
        elif season_name <= "2006-07":
            return "NOK"
        elif season_name <= "2012-13":
            return "NOH"
        else:
            return "NOP"
    elif team_id == 1610612751:  # BRK
        if season_name <= "2011-12":
            return "NJN"
        else:
            return "BRK"
    elif team_id == 1610612760:  # OKC
        if season_name <= "2007-08":
            return "SEA"
        else:
            return "OKC"
    elif team_id == 1610612763:  # MEM
        if season_name <= "2000-01":
            return "VAN"
        else:
            return "MEM"
    elif team_id == 1610612766:  # CHA
        if season_name <= "2001-02":
            return "CHH"
        elif season_name <= "2013-14":
            return "CHA"
        else:
            return "CHO"
    elif team_id == 1610612756:  # PHO
        return "PHO"
    else:
        return find_team_name_by_id(team_id)['abbreviation']


In [65]:
abr_id_map = {}
for team in static.teams.get_teams():
    team_id = team['id']
    for season_name in seasons:
        abr = get_team_abr(team_id, season_name)
        if abr in abr_id_map and abr_id_map[abr] != team_id:
            print("Error", abr, abr_id_map[abr], team_id)
        abr_id_map[abr] = team_id

In [11]:
game = season.schedule.games.final_games[0]
game

{'game_id': '0022000001',
 'date': '2020-12-22',
 'home_team_id': 1610612751,
 'visitor_team_id': 1610612744,
 'status': 'Final'}

In [13]:
# YYYY-MM-DD -> YYYYMMDD
date = "".join(game['date'].split('-'))
date

'20201222'

In [14]:
home_abr = get_team_abr(game['home_team_id'], season.name)
home_abr

'BRK'

In [15]:
def get_game_url(game):
    date = "".join(game['date'].split('-'))
    home_abr = get_team_abr(game['home_team_id'], season.name)
    url = f"https://www.basketball-reference.com/boxscores/{date}0{home_abr}.html"
    return url

In [16]:
url = get_game_url(game)
url

'https://www.basketball-reference.com/boxscores/202012220BRK.html'

In [19]:
page = requests.get(url, timeout=15)


In [20]:
soup = BeautifulSoup(page.text, features="lxml")


In [24]:
home_abr = get_team_abr(game['home_team_id'], season.name)
away_abr = get_team_abr(game['visitor_team_id'], season.name)
home_id = game['home_team_id']
away_id = game['visitor_team_id']


In [50]:
soup.find(id="content").find_all("div", recursive=False)[-2].find("strong")


<strong>Inactive: </strong>

In [92]:
game_stats = soup.find(id="content").find_all(
    "div", recursive=False)[-2]  # .contents[-8]
game_stats


<div>
<div><strong>Inactive: </strong><span><strong>GSW</strong> </span><a href="/players/g/greendr01.html">Draymond Green</a>, <a href="/players/m/mannini01.html">Nico Mannion</a>, <a href="/players/s/smailal01.html">Alen Smailagić</a>, <a href="/players/t/thompkl01.html">Klay Thompson</a>   <span><strong>BRK</strong> </span><a href="/players/c/chiozch01.html">Chris Chiozza</a>, <a href="/players/c/claxtni01.html">Nic Claxton</a>   </div>
<div><strong>Officials: </strong><a href="/referees/bucheni99r.html">Nick Buchert</a>, <a href="/referees/twardsc99r.html">Scott Twardoski</a>, <a href="/referees/zarbaza99r.html">Zach Zarba</a></div>
<div><strong>Time of Game: </strong>2:14</div>
</div>

In [95]:
official_tags = game_stats.find_all('div')[1].find_all('a')
official_tags

[<a href="/referees/bucheni99r.html">Nick Buchert</a>,
 <a href="/referees/twardsc99r.html">Scott Twardoski</a>,
 <a href="/referees/zarbaza99r.html">Zach Zarba</a>]

In [103]:
inactive = game_stats.div.contents
inactive


[<strong>Inactive: </strong>,
 <span><strong>GSW</strong> </span>,
 <a href="/players/g/greendr01.html">Draymond Green</a>,
 ',\xa0',
 <a href="/players/m/mannini01.html">Nico Mannion</a>,
 ',\xa0',
 <a href="/players/s/smailal01.html">Alen Smailagić</a>,
 ',\xa0',
 <a href="/players/t/thompkl01.html">Klay Thompson</a>,
 '\xa0\xa0\xa0',
 <span><strong>BRK</strong> </span>,
 <a href="/players/c/chiozch01.html">Chris Chiozza</a>,
 ',\xa0',
 <a href="/players/c/claxtni01.html">Nic Claxton</a>,
 '\xa0\xa0\xa0']

In [70]:
@dataclass 
class TeamGameData:
    starters: list[int] = field(default_factory=list)
    bench: list[(int, bool)] = field(default_factory=list)
    inactive: list[int] = field(default_factory=list)



In [89]:
@dataclass
class Official:
    name: str
    br_id: str

In [85]:
team_data = { home_id: TeamGameData(), away_id: TeamGameData()}

In [90]:
officials = []

In [88]:
team_data

{1610612751: TeamGameData(starters=[], bench=[], inactive=[1629185, 1629651]),
 1610612744: TeamGameData(starters=[], bench=[], inactive=[203110, 1630185, 1629346, 202691])}

In [87]:
current_team = None
for tag in inactive:
    if tag.name == 'span':
        abr = tag.strong.string
        current_team = abr_id_map[abr]
        ros = season.team_seasons[current_team].roster
    elif tag.name == 'a':
        br_id = tag['href'].split('/')[-1][:-5]
        try:
            id = ros[ros.br_id == br_id].id.item()
        except ValueError:
            # for players who haven't played any games this season, like Klay
            name = tag.string
            id = season.team_seasons[current_team].roster_df[season.team_seasons[current_team].roster_df.PLAYER == name].PLAYER_ID.item()
        team_data[current_team].inactive.append(id)


In [100]:
def seconds_from_time(time: str) -> int:
    """
    Return the amount of seconds represented by str time

    Args:
        time (str): time in 'm:ss' or 'mm:ss' format

    Returns:
        int:
    """
    [min, sec] = time.split(':')
    return int(min) * 60 + int(sec)


In [97]:
for tag in official_tags:
    br_id = tag['href'].split('/')[-1][:-5]
    name = tag.string
    officials.append(Official(name, br_id))
officials

[Official(name='Nick Buchert', br_id='bucheni99r'),
 Official(name='Scott Twardoski', br_id='twardsc99r'),
 Official(name='Zach Zarba', br_id='zarbaza99r')]

In [101]:
for team_id in [home_id, away_id]:
    team_abr = get_team_abr(team_id, season.name)
    rows = soup.find(id=f"box-{team_abr}-game-basic").tbody.find_all('tr')
    ros = season.team_seasons[team_id].roster
    
    for row in rows[:5]:
        # starters
        br_id = row.find("th")['data-append-csv']
        id = ros[ros.br_id == br_id].id.item()
        team_data[team_id].starters.append(id)
        
    for row in rows[6:]:
        # bench
        br_id = row.find("th")['data-append-csv']
        id = ros[ros.br_id == br_id].id.item()
        did_play = row.find("td")['data-stat'] != 'reason'
        team_data[team_id].bench.append((id, did_play))
        
    


In [102]:
team_data


{1610612751: TeamGameData(starters=[202681, 201142, 203925, 203915, 201599, 202681, 201142, 203925, 203915, 201599], bench=[1627747, 1629013, 1628386, 201145, 1627752, 1629617, 1628971, 1627789, 1629066, 204020, (1627747, True), (1629013, True), (1628386, True), (201145, True), (1627752, True), (1629617, True), (1628971, True), (1627789, True), (1629066, True), (204020, True)], inactive=[1629185, 1629651]),
 1610612744: TeamGameData(starters=[203952, 201939, 1626162, 1630164, 1629672, 203952, 201939, 1626162, 1630164, 1629672], bench=[202954, 1629673, 1629308, 1627737, 1627814, 203145, 1626172, 1628539, (202954, True), (1629673, True), (1629308, True), (1627737, True), (1627814, True), (203145, True), (1626172, True), (1628539, True)], inactive=[203110, 1630185, 1629346, 202691])}

In [116]:
season_name = "2000-01"
season_name[:2]+season_name[5:]


'2001'

In [120]:
teams.find_team_by_abbreviation('WAS')

AttributeError: 'dict_values' object has no attribute 'find_team_by_abbreviation'

In [78]:

ds = "April 27, 1977"
datetime.datetime.strptime(ds, "%B %d, %Y")


datetime.datetime(1977, 4, 27, 0, 0)

In [38]:
players.find_player_by_id(78197)


{'id': 78197,
 'full_name': 'Michael Smith',
 'first_name': 'Michael',
 'last_name': 'Smith',
 'is_active': False}

In [39]:
start = 0
begin_date = "2000-05-01"
end_date = "2000-07-01"
url = f"http://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate={begin_date}&EndDate={end_date}&PlayerMovementChkBx=yes&Submit=Search&start={str(start)}"
html = urlopen(url)
soup = BeautifulSoup(html)
# soup.find_all




In [40]:
transactions = []
table_rows = soup.find_all('div')[3].table.find_all('tr')
if len(table_rows) == 1:
    print("no more")
for trans in table_rows[1:]:
    trans_data = trans.find_all('td')

    date = trans_data[0].contents[0]

    team = trans_data[1].contents[0].strip()

    acquired = trans_data[2].contents[0]
    if acquired:
        acquired = acquired[3:]

    relinquished = trans_data[3].contents[0]
    if relinquished:
        relinquished = relinquished[3:]

    notes = trans_data[4].contents[0]

    transactions += (date, team, acquired, relinquished, notes)

In [56]:
dir = "C:/Users/bhalb/nbaproject/data/transactions1999.csv"

df= pd.DataFrame(trans, columns=["date", "team", "acquired", "relinquished", "notes"])
df.to_csv(dir, index=False)


In [120]:
def process_name(name: str):
    name = name.strip()
    if name and name[0] == "•":
        name = name[2:]
    return name

def get_player_transactions(year: int):
    begin_date = f"{str(year)}-01-01"
    end_date = f"{str(year)}-12-31"
    url = f"http://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate={begin_date}&EndDate={end_date}&PlayerMovementChkBx=yes&Submit=Search&start="
    start = 0
    
    transactions = []

    while True:
        # print("Start: ", start)
        current_url = url + str(start)
        start += 25

        html = urlopen(current_url)
        soup = BeautifulSoup(html)
        table_rows = soup.find_all('div')[3].table.find_all('tr')
        if len(table_rows) == 1:
            break
        
        for trans in table_rows[1:]:
            trans_data = trans.find_all('td')

            date = trans_data[0].contents[0]

            team = trans_data[1].contents[0].strip()

            if trans_data[2].contents[0]:
                acquired = list(map(process_name, trans_data[2].contents[::2]))
            else:
                acquired = []
            # if acquired:
            #     acquired = acquired[3:]

            if trans_data[3].contents[0]:
                relinquished = list(map(process_name, trans_data[3].contents[::2]))
            else:
                relinquished = []
            # if relinquished:
            #     relinquished = relinquished[3:]

            notes = trans_data[4].contents[0]

            transactions.append((date, team, acquired, relinquished, notes))
            
    return transactions



In [105]:
trans = get_player_transactions(2000)


In [111]:
def save_transaction_data(year):
    path = f"C:/Users/bhalb/nbaproject/data/transactions{str(year)}.json"
    trans = get_player_transactions(year)
    df = pd.DataFrame(
        trans, columns=["date", "team", "acquired", "relinquished", "notes"])
    df.to_json(path)


In [123]:
for i in range(2001, 2022):
    save_transaction_data(i)

In [12]:
def indicates_player_signing(note: str):

    player_signing_starts = (
        "waived",
        "signed",
        "re-signed",
        "resigned",
        "re-resigned",
        "deceased",
        "player became a free agent",
        "contract expired",
        "claimed off waivers",
        "retired",
        "reached agreement to buy out contract",
        "agreed to contract buyout",
        "player agreed to waive",
        "announced retirement",
        "team announced that player will retire",
        "player exercised",
        "team exercised",
        "player declined",
        "team declined",
        "player opted out",
        "player became",
        "exercised contract option",
        "declined to exercise team option",
        "player elected",
        "lost free agent",
        "lost restricted free agent",
        "lost restricted free agnet",
        "lost restrictred",
        # "renounced rights",
        "team rescinded",
        "contract voided by NBA",
        "un-retired and signed",
        "player reneged on contract",
        "player did not report to training camp",
        "reached a $1.5M buyout agreement",
        "reached a buyout agreement",
        "team bought out contract",
        "terminated contract",
        "team tendered qualifying offer",
        "extended qualifying offer",
        "team withdrew qualifying offer",
        "team extended qualifying offer",
        "player declared free agency",
        "became a free agent",
        "declared free agency",
        "lost unrestricted free agent",
        "released",
        "team terminated contract",
        "lost in 2004 NBA expansion draft",
        "2004 NBA expansion draft pick",
        "became an unrestricted free agent",
        "contract voided",
        "player signed",
    )

    # not re-signed as head coach
    if note.startswith(player_signing_starts):
        return True

    return False

def indicates_ignore(note: str):
    ignore_starts = (
        "agreed to terms",
        "agreed to trems",
        "failed physical",
        "re-signed as",
        "resigned as",
        "retired as",
        "resigned/retired as",
        "re--signed as",
        "hired",
        "fired",
        "transfer of ownership",
        "purchased team",
        "purchase of team",
        "promoted to",
        "re-assigned",
        "replaced as",
        "retained as",
        "extended contract",
        "named",
        "demoted",
        "moved from",
        "resumed role",
        "NBA approved purchase of team",
        "agreed to sell team",
        "agreed to purchase team",
        "reached agreement for transfer of ownership",
        "expansion committee",
        "assistant coach",
        "special assistant coach",
        "agreed to re-sign as head coach",
        "released assistant",
        "interim",
        "promoted",
        "team added",
        "added to",
        "London arbitrator",
        "arbitrator",
        "exercised their option to claim",
        "Announced they will exercise the option on",
        "announced he will return as head coach",
        "formally re-signed as head coach",
        "contract as head coach",
        "ammended 8/12/01 trade",
        "awarded by NBA",
        "activated from IL",
        "reached contract termination settlement",
        "NBA Board",
        "NBA approved sale",
        "team used amnesty",
        "agreed to step down",
        "contract altered",
        "compensation",
        "purchased",
        "(re-?)signed as",
        "declined to",
        "reportedly",
        "renounced their traded player exceptions",
        "renounced",
        "renoucned",
        "reassigned",
        "2000 NBA draft",
        "2001 NBA draft",
        "2002 NBA draft",
        "2003 NBA draft",
        "2004 NBA draft",
        "2005 NBA draft",
        "2006 NBA draft",
        "2007 NBA draft",
        "2008 NBA draft",
        "2009 NBA draft",
        "2010 NBA draft",
        "2011 NBA draft",
        "2012 NBA draft",
        "2013 NBA draft",
        "2014 NBA draft",
        "2015 NBA draft",
        "2016 NBA draft",
        "2017 NBA draft",
        "2018 NBA draft",
        "2019 NBA draft",
        "2020 NBA draft",
        "2021 NBA draft",
    )
    # not re-signed as head coach

    if note.startswith(ignore_starts):
        return True

    return False

def indicates_trade(note: str):
    trade_starts = (
        "trade",
        "announced trade",
        "3-team trade",
        "4-team trade",
        "5-team trade",
        "6-team trade",
        "earlier trade with",
        "sent to", # just picks?
        "received from",
    )
    # rights to
    # 2000
    if note.startswith(trade_starts):
        return True
    return False

In [13]:
# import csv

year = 2007
path = f"C:/Users/bhalb/nbaproject/data/transactions{str(year)}.json"
# with open(path, newline='') as csvfile:
#     reader = csv.reader(csvfile, 
df = pd.read_json(path)
season_start = "2000-10-31"
for row in df.itertuples():
    note = row.notes.strip()
    if indicates_ignore(note):
        continue
    if indicates_player_signing(note):
        continue
    if indicates_trade(note):
        continue
    print(note, row.acquired, row.relinquished)
    # TODO when player added to team, remove from old team
    break
    if len(row.acquired) > 1:
        
        break
    # if d.date >= season_start:
    #     if 
    #     print(d)
    #     break


microfracture surgery on right knee (out for season) ['Sean May'] ['']


In [ ]:
def getTeamRatings():
    url = 'https://www.basketball-reference.com/leagues/NBA_2020_ratings.html'
    html = urlopen(url)
    soup = BeautifulSoup(html)
    headers = [th.getText() for th in soup.findAll('tr')[1].findAll('th')]
    rows = soup.findAll('tr') # 2 is first team
    team_stats = [[i-1]+[td.getText() for td in rows[i].findAll('td')] for i in range(2,len(rows))]
    stats = pd.DataFrame(team_stats, columns = headers)
    
    return stats

stats = getTeamRatings()
stats.head(30)

In [ ]:
# get lineup data
url = 'https://stats.nba.com/lineups/advanced/?Season=2018-19&SeasonType=Regular%20Season'
html = urlopen(url)
soup = BeautifulSoup(html)
#headers = [th.getText() for th in soup.findAll('tr')[1].findAll('th')]
soup